In [4]:
import recordlinkage
from Annotation_helper import *
import numpy as np

In [5]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")

In [17]:
transactions.columns

Index(['Unnamed: 0', 'ID', 'Datum', 'VerkoperVoornaam', 'VerkoperPatroniem',
       'VerkoperTussen', 'VerkoperAchternaam', 'VerkoperGender',
       'VerkoperHerkomst', 'VerkoperBeroep', 'VerkoperWerkplek',
       'VerkoperEtniciteit', 'VerkoperStatus', 'KoperVoornaam',
       'KoperPatroniem', 'KoperTussen', 'KoperAchternaam', 'KoperGender',
       'KoperHerkomst', 'KoperBeroep', 'KoperWerkplek', 'KoperEtniciteit',
       'KoperStatus', 'SlaafNaamNieuw', 'SlaafNaamOrigineel', 'SlaafTaalNaam',
       'SlaafKaste', 'SlaafGender', 'SlaafLeeftijd', 'Opmerking', 'Entry-ID'],
      dtype='object')

In [10]:
permissions.columns

Index(['ID', 'Permissie', 'Inventarisnummer', 'Scan', 'Plaats', 'Datum',
       'BezitterVoornaam', 'BezitterPatroniem', 'BezitterTussen',
       'BezitterAchternaam', 'BezitterGender', 'BezitterHerkomst',
       'BezitterBeroep', 'BezitterWerkplek', 'BezitterEtniciteit',
       'BezitterStatus', 'SlaafNaamNieuw', 'SlaafNaamOrigineel', 'SlaafKaste',
       'SlaafGender', 'Opmerking', 'Betaling', 'Door[Kantlijn]'],
      dtype='object')

In [16]:
permissions[permissions["BezitterEtniciteit"].notna()]

,ID,Permissie,Inventarisnummer,Scan,Plaats,Datum,BezitterVoornaam,BezitterPatroniem,BezitterTussen,BezitterAchternaam,...,BezitterWerkplek,BezitterEtniciteit,BezitterStatus,SlaafNaamNieuw,SlaafNaamOrigineel,SlaafKaste,SlaafGender,Opmerking,Betaling,Door[Kantlijn]
24,897-1-25,25,897,1,Cochin,[…]-05-09,David,NaN,NaN,More,...,Schip Peintroke,Engels,NaN,Joan,NaN,NaN,m,NaN,M[its betalende ge]regtigheid aan den pagter,den vaandrig der burgerije M: Lavier
106,897-5-8,8,897,5,Cochin,1770-10-29,Juhn,NaN,NaN,Meken,...,NaN,Engelsch,NaN,October,NaN,NaN,m,NaN,NaN,NaN
107,897-5-9,9,897,5,Cochin,1770-10-29,Juhn,NaN,NaN,Meken,...,NaN,Engelsch,NaN,October,NaN,NaN,m,NaN,NaN,NaN
108,897-5-10,10,897,5,Cochin,1770-11-06,Paxsou,NaN,NaN,NaN,...,NaN,Engelsch,NaN,Roza,NaN,NaN,v,NaN,Enz:,Den herbergier Groenroode gekogt
109,897-5-11,11,897,5,Cochin,1770-11-06,Kepp,NaN,NaN,NaN,...,NaN,Engelsch,NaN,Thome,NaN,NaN,m,NaN,Enz:,Daniel Bakker verkogt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,1333-242-3,3,1333,242,Cochin,1795-05-01,NaN,NaN,NaN,NaN,...,NaN,Deens,NaN,Roza,NaN,NaN,v,NaN,Gerechtigheid,NaN
3920,1333-242-4,4,1333,242,Cochin,1795-05-01,Douglas,NaN,NaN,NaN,...,NaN,Engels,NaN,Antoni,NaN,NaN,m,NaN,Gerechtigheid,['...]
3922,1333-243-1,1,1333,243,Cochin,1795-05-01,Jean,NaN,NaN,Jasjean,...,NaN,Frans,NaN,Dalida,NaN,NaN,v,NaN,Gerechtigheid,NaN
3923,1333-243-2,2,1333,243,Cochin,1795-05-04,Douglas,NaN,NaN,NaN,...,NaN,Engels,NaN,Flo[...],NaN,NaN,NaN,NaN,Gerechtigheid,NaN


In [11]:
permissions["BezitterEtniciteit"].unique()

array([nan, 'Engels', 'Engelsch', '[Engel]sch', 'Jood', 'Fransch',
       'Portugees', 'Armeender', 'Engelsche', 'Moor', 'Engelsman',
       'Maleijs', 'Toepas', 'Joods', 'Armeens', 'Deens', 'Portugeesch',
       'Frans', 'Deen', 'Englesch', 'Armenijer', 'Armeniers', 'Arabier',
       '[Fransch]', 'Armeensch', '[Por]tugeesch', 'Armeendersch',
       'Fransman', 'Deensch', 'Italiaansch', 'Moorsch', 'Malaitsch',
       'Het schip Teijlingen'], dtype=object)

In [18]:
dist_metric = 'levenshtein'
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(permissions, transactions)

compare_cl = recordlinkage.Compare()

compare_cl.exact('SlaafGender', 'SlaafGender', label='SlaafGender')
compare_cl.string('SlaafNaamNieuw', 'SlaafNaamNieuw', label='SlaafNaamNieuw', method=dist_metric)
compare_cl.exact('BezitterGender', 'KoperGender', label='BezitterGender')
compare_cl.string('BezitterVoornaam', 'KoperVoornaam', label='BezitterVoornaam', method=dist_metric)
compare_cl.string('BezitterTussen', 'KoperTussen', label='BezitterTussen', method=dist_metric)
compare_cl.string('BezitterAchternaam', 'KoperAchternaam', label='BezitterAchternaam', method=dist_metric)
compare_cl.string('BezitterBeroep', 'KoperBeroep', label='BezitterBeroep', method=dist_metric)
compare_cl.string('BezitterStatus', 'KoperStatus', label='BezitterStatus', method=dist_metric)
compare_cl.string('BezitterEtniciteit', 'KoperEtniciteit', label='BezitterEtniciteit', method=dist_metric)
n_batches = 100
split_perm = np.array_split(permissions, n_batches)

In [20]:
for i, p in enumerate(split_perm):
    print(f"[{i}/{n_batches}]")
    candidate_links = indexer.index(p, transactions)
    features = compare_cl.compute(candidate_links, p, transactions)
    features = features.reset_index()
    features = features.rename(columns={"level_0": "permission_indx", "level_1": "transaction_indx"})
    features.to_csv(f"filtered_Distances/distances_{i}.csv", index=False)
    clear_output(wait=True)

[99/100]
